In [ ]:
#default_exp indexer

In [ ]:
#hide
from nbdev.showdoc import *

# Indexer

Given a dataset of tensors, returns a dictionary archive and a treemap structure (and saves them to disk)

## Joiner

This executor `needs` both Encoder and Loader to send it the new and old vectors, respectively. So it needs to be preceded by the **join_all** component to make sure we're not missing new data before handing it over to the indexer -- or indexing old data that no longer exists!

In [ ]:
#export
def join_all(db, new_files, new_embeddings):
    start = len(db)
    for i, file in enumerate(new_files):
        path, slug = file
        index = i + start
        db[slug] = {
            'slug': slug,
            'fpath': path,
            'embed': new_embeddings[i],
            'index': index
        }
    return(db)

In [ ]:
import torch
from pathlib import Path
from memery.loader import db_loader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
root = Path('images/')

In [ ]:
db = db_loader(root/'memery.pt',device)

In [ ]:
[o[0] for o in db.items()][:5]

['Wholesome-Meme-3_1621298477',
 'Wholesome-Meme-44_1621298480',
 'Wholesome-Meme-69_1621298481',
 'Wholesome-Meme-59_1621298480',
 'Wholesome-Meme-68_1621298481']

In [ ]:
len(db)

78

Building treemap takes a long time. I don't think `annoy` uses the GPU at all?

In [ ]:
#export
from annoy import AnnoyIndex

In [ ]:
#export
def build_treemap(db):
    treemap = AnnoyIndex(512, 'angular')
    for v in db.values():
        treemap.add_item(v['index'], v['embed'])

    # Build the treemap, with 5 trees rn
    treemap.build(5)

    return(treemap)
    

In [ ]:
t = build_treemap(db)

In [ ]:
t.get_n_items(), t.get_n_trees()

(79, 5)

In [ ]:
#export
import torch

In [ ]:
#export
def save_archives(root, treemap, db):
    dbpath = root/'memery.pt'
    if dbpath.exists():
#         dbpath.rename(root/'memery-bak.pt')
        dbpath.unlink()
    torch.save(db, dbpath)
    
    treepath = root/'memery.ann'
    if treepath.exists():
#         treepath.rename(root/'memery-bak.ann')
        treepath.unlink()
    treemap.save(str(treepath))
    
    return(str(dbpath), str(treepath))

In [ ]:
save_archives(root, t, db)

('images/memery.pt', 'images/memery.ann')